In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
import tqdm
import torch.nn.functional as F
import tqdm
import impute
import PMLE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import datetime

/Users/remycanario/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)
Using TensorFlow backend.


# Data Preprocessing

In [28]:
df = pd.read_csv('testing_df.csv',index_col=0)
df.drop('TransactionID',axis=1,inplace=True)


/Users/remycanario/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1. Prune Columns

In [46]:
dropped_cols = []

#drop columns that have excessive NA 
for col in df.columns:
    if df[col].isnull().sum()>0.95*df.shape[0]:
        df.drop(col,axis=1,inplace=True)
        dropped_cols.append(col)
        
#drop columns that have only one value
for col in df.min()[df.min()==df.max()].index:
    df.drop(col,axis=1,inplace=True)
    dropped_cols.append(col)

## 2. Feature Engineering

In [51]:
X = df.drop('isFraud',axis=1)
y = df['isFraud']

#Create feature for number of NAs per column
row_nas = []
for i in range(X.shape[0]):
    row_nas.append(X.iloc[i,:].isna().sum())
X['NAs']= row_nas

#Create features of characteristics of transaction amounts
X['amtby5']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[0]) % 5==0 else 0)
X['centsby5']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1]) % 5==0 else 0)
X['threedec']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1]) > 99 else 0)
X['centsnot0']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1][:2])!=0 else 0)
X['centsnot95']=X.TransactionAmt.apply(lambda x: 1 if int(str(x).split('.')[1][:2])!=95 else 0)
X['digits']= X.TransactionAmt.apply(lambda x: str(x).split('.')[0][-1])
X['places']= X.TransactionAmt.apply(lambda x: str(len(str(x).split('.')[0])))


#Prepare card columns for one hot encoding 
X.loc[:,'card1':'card5'].astype(str)

#Convert TransactionID into datetime and derive features
X['Datetime'] = X.TransactionDT.apply(lambda x:datetime.datetime(2018,1,1)+datetime.timedelta(seconds=x))
X['Weekday']= X.Datetime.apply(lambda x: x.weekday())
X['Month']= X.Datetime.apply(lambda x: x.month)
X['Hour']= X.Datetime.apply(lambda x: x.hour)
X['Day']=(X.TransactionDT - 86400)//(60*60*24)
X['Week']= X.Day//7
X['Minute']= X.Datetime.apply(lambda x: x.minute)
X.drop('Datetime',axis=1,inplace=True)

X = pd.get_dummies(X,drop_first=True,dummy_na=True)

## 3. Train Test Split

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=.2,random_state=0)

## 4. Fill Missing Values with Multiple Imputation

In [6]:
#Establish floors and ceilings for column values
column_ranges = {}
for col in X_train.columns:
    column_ranges[col]=(X_train[col].min(),X_train[col].max())

#Impute missing values using multiple imputation
X_train, train_problems = impute.impute_missing_values(X_train,col_floor_ceiling_dict=column_ranges)
X_val, val_problems = impute.impute_missing_values(X_val,col_floor_ceiling_dict=column_ranges)

print('X_train NAs = ', X_train.isna().sum().sum())
print('X_val NAs = ', X_train.isna().sum().sum())

  0%|          | 0/342 [00:00<?, ?it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  0%|          | 1/342 [00:01<08:03,  1.42s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  1%|          | 2/342 [00:02<07:32,  1.33s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

  6%|▋         | 22/342 [00:22<05:01,  1.06it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  7%|▋         | 23/342 [00:23<05:03,  1.05it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
  7%|▋         | 24/342 [00:24<05:08,  1.03it/s]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 13%|█▎        | 44/342 [00:45<06:00,  1.21s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 13%|█▎        | 45/342 [00:46<05:58,  1.21s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 13%|█▎        | 46/342 [00:47<05:39,  1.15s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 19%|█▉        | 66/342 [01:11<05:09,  1.12s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 20%|█▉        | 67/342 [01:12<05:29,  1.20s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 20%|█▉        | 68/342 [01:13<05:11,  1.14s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

 26%|██▌       | 88/342 [01:37<04:49,  1.14s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 26%|██▌       | 89/342 [01:38<04:47,  1.14s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 26%|██▋       | 90/342 [01:39<04:42,  1.12s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

...

 67%|██████▋   | 229/342 [02:55<03:39,  1.95s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 67%|██████▋   | 230/342 [02:56<03:16,  1.76s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 68%|██████▊   | 231/342 [02:59<03:41,  2.00s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 73%|███████▎  | 251/342 [03:25<01:48,  1.19s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 74%|███████▎  | 252/342 [03:27<01:48,  1.21s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 74%|███████▍  | 253/342 [03:28<02:02,  1.38s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 80%|███████▉  | 273/342 [03:58<02:03,  1.78s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 80%|████████  | 274/342 [03:59<01:59,  1.76s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 80%|████████  | 275/342 [04:02<02:05,  1.88s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 86%|████████▋ | 295/342 [04:32<01:21,  1.74s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 87%|████████▋ | 296/342 [04:33<01:11,  1.56s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 87%|████████▋ | 297/342 [04:35<01:13,  1.63s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

 93%|█████████▎| 317/342 [05:07<00:35,  1.42s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 93%|█████████▎| 318/342 [05:08<00:32,  1.36s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][na_index] = preds
 93%|█████████▎| 319/342 [05:09<00:30,  1.31s/it]/Users/remycanario/Documents/DataScience/credit-card-fraud/impute.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

## 5. Standardize

In [57]:
#Save meta data
columns = X_train.columns
normal_inds = y_train[y_train==0].reset_index().index

#Standardize dfs
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_val_sc = sc.fit_transform(X_val)

# Autoencoder Anomaly Detection
### With undercomplete, regularized and denoising autoencoders using pytorch

## Data Processing

In [58]:
n_features = X_train_sc.shape[1]

#Convert non-fraud rows to pytorch tensor
normal_train = X_train_sc[normal_inds,:]
normal_torch = torch.from_numpy(normal_train).type(torch.FloatTensor)

#Convert train and val sets to pytorch tensor
train_torch = torch.from_numpy(X_train_sc).type(torch.FloatTensor)
val_torch = torch.from_numpy(X_val_sc).type(torch.FloatTensor)

## Create Pytorch objects

In [11]:
class AutoEncoder(nn.Module):
    
    def __init__(self,n_features,hidden_nodes,dropout=None):
        
        '''PARAMATERS
           n_features: number of X variables in dataset
           hidden_nodes: number of nodes in hidden layer
           dropout: fraction of nodes to dropout (0 < dropout <1)'''
        
        super(AutoEncoder, self).__init__()
        self.n_features=n_features
        self.n_hidden = hidden_nodes
        self.encoder = nn.Linear(n_features,hidden_nodes)
        self.decoder = nn.Linear(hidden_nodes,n_features)
        self.output_layer = nn.Linear(n_features,n_features)
        self.dropout = dropout
        
        
    def forward(self,x):
        if self.dropout!=None:
            x = F.relu(F.dropout(self.encoder(x)))
        else:
            x = F.relu(self.encoder(x))
        self.hidden_layer=x
        x = F.relu(self.decoder(x))
        x = self.output_layer(x)
        return x
    

In [69]:
def train_autoencoder(model, dataset, loss_func, optimizer, epochs=100, batch_size=1024, 
                      noise_factor=None, random_seed=None):
    '''Parameters
       model: instantiated autoencoder
       dataset: numpy array of X variables
       loss_func: instantiated loss function
       optimizer: instantiated optimizer
       epochs: number of epochs
       batch_size: batch_size
       noise_factor: magnitude of noise added to data
         for a denoising autoencoder (0 < noise_factor <=1)
       random_seed: random_seed'''
    
    if random_seed!=None:
        torch.manual_seed(random_seed)
    train_loader = torch.utils.data.DataLoader(dataset, 
                                               batch_size=batch_size, 
                                               shuffle=True)
    readout_batch_interval = ((batch_size/1024)*50)//1
    
    for epoch in range(epochs):
        counter = 0
        print('\n\033[1mEpoch {}\033[0m\n'.format(epoch+1))
        for batch in train_loader:
            
            if noise_factor!=None:
                batch = batch + noise_factor * torch.randn(*batch.shape)
            batch = torch.autograd.Variable(batch)
            optimizer.zero_grad()
            recon = model(batch)
            loss = loss_func(recon, batch)
            
            if (counter%readout_batch_interval==0):
                print('Batch {} Loss: {:.4f}'.format(counter, float(loss)))
            loss.backward()
            optimizer.step()
            counter+=1
        epoch_loss = loss_func(model(dataset), dataset)
        print('\nEPOCH {} LOSS: {:.4f}'.format(epoch+1, float(epoch_loss)))

## Train Autoencoders

### Autoencoder I: Dropout

#### Instantiate encoder and loss function

In [72]:
ae1 = AutoEncoder(n_features,int(n_features*1.5//1),dropout=0.3)
loss_func = nn.MSELoss()

#### Train autoencoder

In [ ]:
optimizer = torch.optim.SGD(ae1.parameters(), lr=0.2, momentum=0.9,nesterov=True)
train_autoencoder(model=ae1,
                   dataset=normal_torch,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=256,
                   epochs=600)


Epoch 1

Batch 0 Loss: 0.7022
Batch 12 Loss: 0.9630
Batch 24 Loss: 0.9095
Batch 36 Loss: 1.2853

EPOCH 1 LOSS: 0.8976

Epoch 2

Batch 0 Loss: 1.0781
Batch 12 Loss: 0.7349
Batch 24 Loss: 0.7180
Batch 36 Loss: 1.0202

EPOCH 2 LOSS: 0.8653

Epoch 3

Batch 0 Loss: 1.0308
Batch 12 Loss: 0.7727
Batch 24 Loss: 0.7930
Batch 36 Loss: 0.8226

EPOCH 3 LOSS: 0.8319

Epoch 4

Batch 0 Loss: 0.9719
Batch 12 Loss: 0.8434
Batch 24 Loss: 0.9084


#### Save results

In [ ]:
output1 = ae(normal_train)
train_output1 = ae(train_torch)
val_output1 = ae(val_torch)

#### Inspect validation set reconstruction score

In [ ]:
reconstruction error = np.power(val_output1.numpy() - X_val_sc, 2)
np.corrcoef(y_val,reconstruction_error)

### Autoencoder II: Undercomplete

#### Instantiate autoencoder and loss_function

In [ ]:

ae2 = AutoEncoder(n_features,int(n_features*0.8)//1)
loss_func = nn.MSELoss()

#### Train autoencoder

In [ ]:
optimizer = torch.optim.SGD(ae2.parameters(), lr=0.1,momentum=0.9,nesterov=True)
train_autoencoder(model=ae2,
                   dataset=output1,
                   loss_func=loss_func,
                   optimizer=optimizer,
                   batch_size=1024,
                   epochs=120)

#### Save results

In [ ]:
output2 = ae(output1)
train_output2 = ae(train_torch1)
val_output2 = ae(val_torch1)

#### Inspect validation set reconstruction score

In [ ]:
reconstruction error = np.power(val_output2.numpy() - X_val_sc, 2)
np.corrcoef(y_val,reconstruction_error)

### Autoencoder III: Denoising

In [ ]:
ae3 = AutoEncoder(n_features,n_features)
loss_func = nn.MSELoss()

In [ ]:
optimizer = torch.optim.Adam(ae3.parameters(), lr=0.2)
train_autoencoder(model=ae3,
                  dataset=output2,
                  loss_func=loss_func,
                  optimizer=optimizer,
                  batch_size=512,
                  epochs=100,
                  noise=True,
                  noise_factor=0.25)

In [ ]:
#get outputs
output3 = ae(output2)
train_output3 = ae(train_torch2)
val_output3 = ae(val_torch2)

In [ ]:
#test anomaly detection
train_reconstruction error = np.power(train_output3.numpy() - X_train_sc, 2)
val_reconstruction error = np.power(val_output3.numpy() - X_val_sc, 2)

# Penalized Maximum Likelihood Estimation
Using Firth's Logistic Regression with Intercept Correction

In [ ]:
FLIC = PMLE.Firth_Logit(FLIC=True)
FLIC.fit(X_train_sc,y_train)
FLIC_train_preds = FLIC.predict_proba(X_train_sc)
FLIC_val_preds = FLIC.predict_proba(X_train_sc)

# SMOTE Dataset Resampling
Estimation with XGBoost

## Perform Synthetic Minority Over-Sampling Technique

In [ ]:
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train_sc,y_train)
X_val_smote, y_val_smote = smote.fit_resample(X_val_sc,y_val)

## Get XGBoost estimate of SMOTE dataset

In [ ]:
#Instantiate XGBoost and grid search
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
xgb_gs = GridSearchCV(xgb,params,cv=3)

#fit model and obtain predictions
xgb_gs.fit(X_train_smote,y_train_smote)

smote_train_preds = xgb_gs.predict_proba(X_train_smote)
smote_val_preds = xgb_gs.predict_proba(X_val_smote)

# Ensemble results

## Create Ensemble DFs

In [ ]:
ensemble_X_train = pd.DataFrame(None)
ensemble_X_train['anomaly_score'] = anomaly_score
ensemble_X_train['FLIC'] = FLIC_train_preds
ensemble_X_train['smote'] = smote_train_preds

ensemble_X_val = pd.DataFrame(None)
ensemble_X_val['anomaly_score'] = val_anomaly_score
ensemble_X_val['FLIC'] = FLIC_val_preds
ensemble_X_val['smote'] = smote_val_preds

## Get results using FLIC 

In [ ]:
FLIC.fit(ensemble_X_train,y_train)
preds = FLIC.predict_proba(ensemble_X_val)